# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [12]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [13]:
import tweepy
import math
import os.path
import pandas as pd
import json
from string import *
from random import shuffle
import sys
from mpmath import *

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***@jhp_nt***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [14]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @jhp_nt

#leitura do arquivo no formato JSON
with open(r'C:\Users\hpaiv\Desktop\Insper\Pass\auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [15]:
#Produto escolhido:
produto = 'Razer'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [16]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [17]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



## Importando a planilha

In [24]:
df = pd.read_excel('./{0}.xlsx'.format(produto),sheet="Treinamento")
df

,Treinamento,definicao
0,gostei de um vídeo do @youtube https://t.co/cr...,1
1,estou participando do sorteio da vti e-sports ...,1
2,"#oferta stick razer panthera - 3,600 pesos\nht...",1
3,rt @flowalts: 🔻sorteo gaming🔻\narticulo de raz...,1
4,gostei de um vídeo @youtube https://t.co/w6zdv...,1
5,gostei de um vídeo @youtube https://t.co/vkhke...,1
6,gostei de um vídeo @youtube https://t.co/hudnm...,0
7,gostei de um vídeo @youtube https://t.co/wzkvq...,1
8,gostei de um vídeo @youtube https://t.co/rhzl6...,1
9,aprovando varios produtos razer &lt;3 😍😻,0


## Transformando o DataFrame em dois (relevante=0 , irrelevante=1)

In [26]:
CR = df.loc[df['definicao'] == 0]
CI = df.loc[df['definicao'] == 1]

In [27]:
LR = []
LI = []
for i in CR:
    LR.append(i)
for i in CI:
    LI.append(i)

In [28]:
PTotal=[]
a=0
for u in df['treinamento']:
    PTotal[]
    a+=1

SyntaxError: invalid syntax (<ipython-input-28-156eb94d44bb>, line 4)

In [29]:
i=0

while (i < len (LI)):
    LI[i]=LI[i].replace("_"," ")
    LI[i]=LI[i].replace("'"," ")
    LI[i]=LI[i].replace("?"," ")
    LI[i]=LI[i].replace("$"," ")
    LI[i]=LI[i].replace("*"," ")
    LI[i]=LI[i].replace("%"," ")
    LI[i]=LI[i].replace("-"," ")
    LI[i]=LI[i].replace("<"," ")
    LI[i]=LI[i].replace(">"," ")
    LI[i]=LI[i].replace("'"," ")
    LI[i]=LI[i].replace('"'," ")
    LI[i]=LI[i].replace("!"," ")
    LI[i]=LI[i].replace("."," ")
    LI[i]=LI[i].replace(","," ")
    LI[i]=LI[i].replace("|"," ")
    LI[i]=LI[i].replace(":"," ")
    LI[i]=LI[i].replace("&"," ")
    LI[i]=LI[i].replace("\("," ")
    LI[i]=LI[i].replace("\)"," ")
    i+=1
    
    
    
i=0

while (i < len (LR)):
    LR[i]=LR[i].replace("_"," ")
    LR[i]=LR[i].replace("'"," ")
    LR[i]=LR[i].replace("?"," ")
    LR[i]=LR[i].replace("$"," ")
    LR[i]=LR[i].replace("*"," ")
    LR[i]=LR[i].replace("%"," ")
    LR[i]=LR[i].replace("-"," ")
    LR[i]=LR[i].replace("<"," ")
    LR[i]=LR[i].replace(">"," ")
    LR[i]=LR[i].replace("'"," ")
    LR[i]=LR[i].replace('"'," ")
    LR[i]=LR[i].replace("!"," ")
    LR[i]=LR[i].replace("."," ")
    LR[i]=LR[i].replace(","," ")
    LR[i]=LR[i].replace("|"," ")
    LR[i]=LR[i].replace(":"," ")
    LR[i]=LR[i].replace("&"," ")
    LR[i]=LR[i].replace("("," ")
    LR[i]=LR[i].replace(")"," ")
    LR[i]=LR[i].replace("+"," ")
    i+=1

In [30]:
# for i in CB:
#      i.strip(" /?<>.!$%*\|;:&'")
# for i in CR:
#      i.strip(" /?<>.!$%*\|;:&'")

In [31]:
SR=[]
for w in LR:
    SR.append(set(w.split()))
PR=[]
for i in SR:
    for y in i:
        PR.append(y)

In [32]:
SI=[]
for k in LI:
    SI.append(set(k.split()))
PI=[]
for q in SI:
    for l in q:
        PI.append(l)

In [33]:
DICR={}
for j in range(len(PR)-1):
    cont1=j
    cont2=0
    for cont1 in range(len(PR)-1):
        if PR[j] == PR[cont1+1]:
            cont2+=1
    DICR[PR[j]]=mpf(cont2)
DICI={}
for j in range(len(PI)-1):
    cont1=j
    cont2=0
    for cont1 in range(len(PI)-1):
        if PI[j] == PI[cont1+1]:
            cont2+=1
    DICI[PI[j]]=mpf(cont2)  

In [34]:
for g in DICR:
    h=DICR[g]/len(PR)
    DICR[g]=h
for t in DICI:
    h=DICI[t]/len(PI)
    DICI[t]=h

In [35]:
#Calculando o P(relevante) e o P(irrelevante)
CRS=mpf(CR.size)
CIS=mpf(CI.size)
PRel=CRS+1/(CRS+CIS)
PIel=CIS+1/(CRS+CIS)

TypeError: cannot create mpf from 196

In [38]:
print (len(DICR))
print (len(DICI))

1
1


In [37]:


print("Total de palavras é:")
print("Total de palavras Irrelevantes é:")
print("Total de palavras Relevantes é:")
print("Total de palavras sem repetição é:")
print("Total de palavras Irrelevantes sem repetição é:")
print("Total de palavras Relevantes sem repetição é:")

Total de palavras é:
Total de palavras Irrelevantes é:
Total de palavras Relevantes é:
Total de palavras sem repetição é:
Total de palavras Irrelevantes sem repetição é:
Total de palavras Relevantes sem repetição é:


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


Treinamento
definicao
['Treinamento', 'definicao']
